In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the data
folder_path = r'D:\School\Nano120\TDS_15000\TDS_data_15000'
data = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        try:
            df = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
            data[filename] = df
        except pd.errors.ParserError:
            print(f"Error in file: {file_path}")

# Combine all the data into a single dataframe
df = pd.concat(data.values())

# Reshape des_flux column
des_flux = df['des_flux'].values.reshape(-1, 300)

# Process detrap_en and def_conc columns
def process_column(df, column_name):
    column = df[column_name]
    column = column.dropna()
    column = column.values.reshape(-1, 4)
    column_mean = np.mean(column, axis=1)
    column = pd.DataFrame(column)
    column_mean = pd.DataFrame(column_mean)
    display(column_mean)
    return column, column_mean

detrap_en, detrap_en_mean = process_column(df, 'detrap_en')
def_conc, def_conc_mean = process_column(df, 'def_conc')

# Add Gaussian noise
noise_array = np.zeros_like(des_flux, dtype=np.ndarray)
sigma_list = np.zeros(len(des_flux), dtype=np.ndarray)
for n in range(len(des_flux)):
    sigma_rand = np.random.uniform(1E14, 1E17)
    spect_wnoise = np.random.normal(des_flux[n], sigma_rand, size=len(des_flux[n]))
    noise_array[n] = spect_wnoise
    sigma_list[n] = str(sigma_rand)
plt.plot(noise_array[4])
plt.show()

# Normalize des_flux
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
des_flux_norm = scaler.fit_transform(des_flux.reshape(-1, 1))
des_flux_norm = pd.DataFrame(des_flux_norm)
des_flux_id = np.arange(0, des_flux_norm.shape[0])
des_flux_id = des_flux_id // 300
des_flux_norm["id"] = des_flux_id

# Check shape and statistics
display(des_flux_norm.mean(), des_flux_norm.std(), des_flux_norm.min(), des_flux_norm.max())

# Extract features and perform feature selection
from statsmodels.tsa.stattools import acf
from tsfresh import extract_features
from tsfresh import select_features 
from tsfresh.utilities.dataframe_functions import impute

features = extract_features(des_flux_norm, column_id="id", column_sort="id")
features_filtered = select_features(features, detrap_en_mean)
features_filtered = impute(features_filtered)

# Calculate autocorrelation for each row of des_flux_norm
# calculate autocorrelation
acf_values = []
for i in range(len(des_flux_norm)):
    #acf_values_i = acf(des_flux_norm.loc[i], nlags=10, fft=False, unbiased=True)
    acf_values_i = acf(des_flux_norm.loc[i], fft=True, nlags=max_lag)
    acf_values.append(acf_values_i)


: 